In [5]:
# !pip install MetaTrader5
import MetaTrader5 as mt5
import pandas as pd
# from tqdm.notebook import tqdm

if not mt5.initialize():
    mt5.shutdown()
mt5.terminal_info()

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2940, maxbars=100000, codepage=0, ping_last=160858, community_balance=0.0, retransmission=0.0, company='FX Choice Limited', name='FXChoice MetaTrader 5', language='English', path='C:\\Program Files\\FXChoice MetaTrader 5', ...)

In [110]:
risk = pd.read_csv('1y_risk.csv')
# risk

In [112]:
# get risk (size and SL dist)


In [114]:
# get risk * denomination * contract size

# denomination 
syms = set(list(risk['sym']) + ['GBPNZD'] + ['EURCAD'])
# syms

In [115]:
ticks = {}
for sym in syms:
    
    # print(sym)
    t = mt5.symbol_info_tick(sym)
    # print(t)
    p = (t.bid + t.ask) / 2
    ticks[sym] = p
    
#     if sym[3:] == 'USD':
#         print(sym, 1)
#     else:
#         print(sym, '1 / ' + sym[3:] + 'USD')

In [116]:
denom = {}
for sym in syms:
    if sym[3:] == 'USD':
        # print(sym, 1)
        denom[sym] = 1
    else:
        if sym[3:] + 'USD' in syms:
            # print(sym, sym[3:] + 'USD')
            denom[sym] = ticks[sym[3:] + 'USD']
        else:
            # print(sym, 'USD' + sym[3:], 1/ticks['USD' + sym[3:]])
            denom[sym] = 1/ticks['USD' + sym[3:]]

In [118]:
# denom

In [119]:
risk = risk.to_dict(orient='records')

In [134]:
# risk * denom * contract

scaled_mvmt = {}

for sym in syms:
    if sym in [_['sym'] for _ in risk]:
        r = list(filter(lambda x: x['sym'] == sym, risk))[0]['avg+std']
        if sym[3:] == 'JPY':
            r = r/100
        if sym[3:] != 'JPY' and sym != 'BTCUSD' and sym != 'XAUUSD':
            r = r/10000
        c = 100 if sym == 'XAUUSD' else 1 if sym == 'BTCUSD' else 100000
        scaled_mvmt[sym] = r * denom[sym] * c

# scaled_mvmt

In [127]:
# account balance
account = mt5.account_info()

# account.balance

btc = mt5.symbol_info_tick('BTCUSD')
btc_mid = (btc.bid + btc.ask) / 2
btc_mid

usd = account.balance * btc_mid * 0.000001

fr = 0.025

dr = usd * fr

In [136]:
size = {}
for sym in syms:
    if sym in [_['sym'] for _ in risk]:
        # print(sym, dr/scaled_mvmt[sym])
        size[sym] = round(dr/scaled_mvmt[sym], 2)

In [137]:
size

{'USDCAD': 0.1,
 'NZDJPY': 0.12,
 'EURAUD': 0.08,
 'GBPAUD': 0.06,
 'CADJPY': 0.13,
 'BTCUSD': 0.03,
 'AUDJPY': 0.1,
 'GBPCAD': 0.07,
 'USDCHF': 0.11,
 'NZDCHF': 0.12,
 'EURUSD': 0.09,
 'CHFJPY': 0.11,
 'GBPUSD': 0.06,
 'XAUUSD': 0.02,
 'USDJPY': 0.13,
 'EURNZD': 0.08,
 'GBPJPY': 0.07,
 'AUDUSD': 0.1,
 'EURJPY': 0.1,
 'AUDCHF': 0.11,
 'EURCHF': 0.13,
 'AUDCAD': 0.13,
 'GBPCHF': 0.07,
 'CADCHF': 0.13,
 'NZDUSD': 0.11,
 'EURGBP': 0.08}

In [149]:
dist = {}
for sym in syms:
    c = 100 if sym == 'XAUUSD' else 1 if sym == 'BTCUSD' else 1000 if sym[3:] == 'JPY' else 100000
    if sym in [_['sym'] for _ in risk]:
        # print(sym, scaled_mvmt[sym] / c)
        if sym[3:] == 'JPY':
            dist[sym] = round(scaled_mvmt[sym] / c, 3)
        else:
            dist[sym] = round(scaled_mvmt[sym] / c, 5)

In [150]:
dist

{'USDCAD': 0.00214,
 'NZDJPY': 0.186,
 'EURAUD': 0.00272,
 'GBPAUD': 0.00335,
 'CADJPY': 0.171,
 'BTCUSD': 861.8,
 'AUDJPY': 0.207,
 'GBPCAD': 0.00327,
 'USDCHF': 0.00197,
 'NZDCHF': 0.00182,
 'EURUSD': 0.00234,
 'CHFJPY': 0.197,
 'GBPUSD': 0.0035,
 'XAUUSD': 9.49,
 'USDJPY': 0.162,
 'EURNZD': 0.0027,
 'GBPJPY': 0.327,
 'AUDUSD': 0.00221,
 'EURJPY': 0.22,
 'AUDCHF': 0.00197,
 'EURCHF': 0.00163,
 'AUDCAD': 0.00163,
 'GBPCHF': 0.00327,
 'CADCHF': 0.00165,
 'NZDUSD': 0.002,
 'EURGBP': 0.0027}

In [ ]:


# # get current working orders

# mt5.orders_get()